In [1]:
import os
import openai
import pandas as pd
import numpy as np
from pubmed import search as pubmed
from tqdm import tqdm
import pickle
import time
import glob
openai.organization = "your_organization"
openai.api_key = "API_key"
#openai.Model.list()

In [7]:
pmids_df = pd.read_excel("chatgpt-input-data/original_tau_pmids.xlsx")
# print(pmids_df)
pmids = pmids_df["pmid"]
unique_pmids = list(set(list(pmids)))
# print(len(unique_pmids))

In [9]:
#Extract abstracts
abstracts = {}
errors = []
for pmid in tqdm(unique_pmids):
   pubmed_uri  = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
   uri = pubmed_uri + 'efetch.fcgi?db=pubmed&id='+str(pmid)+'&retmode=XML'
   pdocs = pubmed.fetch_pubmed_by_id(pubmed_uri, str(pmid))
   # print(" >>>> title =", pubmed.get_title_of_document(pdocs, 0))
   try:
      abstract = pubmed.get_abstract_of_document(pdocs, 0)
      #print("Type of asbtract is", type (abstract))        
      if (type (abstract) is str):
            text = abstract
      if isinstance(abstract, list):
            abstract_ = ""
            # print("ordered dictionary is the abstract")
            try:
               for row in abstract:
                  for key , value in (row.items()):
                        if (key == '#text'):
                           abstract_ = abstract_ + value
               text = abstract_
               # print("Successufully parsed pubmed dictioanry file")          
            except :
               print("error while parsing dictionary)")
      if isinstance(abstract, bytes):
          print(pmid)
          text = abstract.decode()
      abstracts[pmid] = text
   except KeyError:
      print(pmid)
      continue


   
   # with open('abstracts-checked.pkl', 'wb') as handle:
   #  pickle.dump(abstracts, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 256/256 [02:34<00:00,  1.66it/s]


In [2]:
#Re run on non processed abstracts
import glob
import time
path = r"rerun"
file_list = glob.glob(path + "/*.txt")
for file in tqdm(file_list):
   file_name = os.path.basename(file)
   name = os.path.splitext(file_name)[0]
   messages = [ {"role": "system", "content": 'Extract BEL Triples:\n- Your task is to extract Biological Expression Language (BEL) triples from provided text.\n- A BEL triple consists of three components: subject, relation, and object.\nThe subject and object should be entities categorized into specific namespaces, such as MESH for diseases or biological processes, HGNC for genes, GO for gene functions, and CHEBI for chemicals.\n- Each subject or object should be preceded by one of the BEL functions: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n- The relation between subject and object should be chosen from the following list: increase, decrease, positiveCorrelation, negativeCorrelation, analogous, association, biomarkerFor, causesNoChange, directlyDecreases, directlyIncreases, hasActivity, hasComponent, hasComponents, hasMember, hasMembers, increases, isA, orthologous, prognosticBiomarkerFor, rateLimitingStepOf, regulates, subProcessOf, transcribedTo, translatedTo.\n- Please provide the support evidence by including the sentence that contains the extracted BEL triple.\n- Use  Namespaces to unambiguously reference concepts. \n- Associate prefix HGNC for human loci, including protein coding genes, ncRNA genes and pseudogenes.\n- Associate prefix CHEBI for molecular entities focused on small chemical compounds.\n- Associate prefix GO to unify the representation of gene and gene product attributes across all species.\n- Associate prefix MeSH for indexing articles for PubMed.\n- Associate prefix miRBase which is a biological database that acts as an archive of microRNA sequences and annotations before miRNA names.\n- Associate prefix MGI which is official gene symbol for a mouse gene.\n- Associate prefix DO as a standardized ontology for human disease.\n- Before each subject or object, use one of the BEL functions from the following: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n\nExample Triple:\nTriple1:\nsubject: biologicalProcess(GO:"response to fluid shear stress")\nrelation: increases\nobject: rnaAbundance(HGNC:NOS3)\nsupport evidence: NO is generated by endothelial NO synthase (eNOS, or NOS3), the activity of which is increased by fluid shear stress through various mechanisms.\n(Note: Ensure that the extracted BEL triples adhere to the specified format and namespaces.)'} ]
   pmid = name
   pubmed_uri  = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
   uri = pubmed_uri + 'efetch.fcgi?db=pubmed&id='+str(pmid)+'&retmode=XML'
   pdocs = pubmed.fetch_pubmed_by_id(pubmed_uri, str(pmid))
   # print(" >>>> title =", pubmed.get_title_of_document(pdocs, 0))
   try:
      abstract = pubmed.get_abstract_of_document(pdocs, 0)
      # print("Type of asbtract is", type (abstract))        
      if (type (abstract) is str):
            text = abstract
      if isinstance(abstract, list):
            abstract_ = ""
            # print("ordered dictionary is the abstract")
            try:
               for row in abstract:
                  for key , value in (row.items()):
                        if (key == '#text'):
                           abstract_ = abstract_ + value
               text = abstract_
               print("Successufully parsed pubmed dictioanry file")          
            except :
               print("error while parsing dictionary)")
   except KeyError:
            print(KeyError)
   try:
      time.sleep(2)
      temperature = 0
      message = "Extract all BEL triples for each sentence and provide support evidence for each triple from the following :" + text            
      if message:
            messages.append(
            {"role": "user", "content": message},)
            chat = openai.ChatCompletion.create(temperature, model="gpt-4", messages=messages, )
            ''''''
      #model="gpt-3.5-turbo"
      reply = chat.choices[0].message.content
      messages.append({"role": "assistant", "content": reply})
      file = open(str(pmid)+".txt", "w", encoding='utf-8')
      file.write(reply)
      file.close()
   except Exception as e:
      print(e)

 23%|██▎       | 6/26 [04:14<13:58, 41.93s/it]

Successufully parsed pubmed dictioanry file


100%|██████████| 26/26 [20:39<00:00, 47.66s/it]


In [2]:
#Test individual abstract
abstracts = {}
pmid = 10464280
pubmed_uri  = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
uri = pubmed_uri + 'efetch.fcgi?db=pubmed&id='+str(pmid)+'&retmode=XML'
pdocs = pubmed.fetch_pubmed_by_id(pubmed_uri, str(pmid))
# print(" >>>> title =", pubmed.get_title_of_document(pdocs, 0))
try:
   abstract = pubmed.get_abstract_of_document(pdocs, 0)
   # print("Type of asbtract is", type (abstract))        
   if (type (abstract) is str):
         text = abstract
   if isinstance(abstract, list):
         abstract_ = ""
         # print("ordered dictionary is the abstract")
         try:
            for row in abstract:
               for key , value in (row.items()):
                     if (key == '#text'):
                        abstract_ = abstract_ + value
            text = abstract_
            # print("Successufully parsed pubmed dictioanry file")          
         except :
            print("error while parsing dictionary)")
   abstracts[pmid] = text
except KeyError:
   print(KeyError)

for i in tqdm(range(10)):
   try:
      messages = [ {"role": "system",
 "content": 'Extract BEL Triples:\n- Your task is to extract Biological Expression Language (BEL) triples from provided text.\n- A BEL triple consists of three components: subject, relation, and object and support sentence.\nThe subject and object should be entities categorized into specific namespaces, such as MESH for diseases or biological processes, HGNC for genes, GO for gene functions, and CHEBI for chemicals.\n- Each subject or object should be preceded by one of the BEL functions: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n- The relation between subject and object should be chosen from the following list: increase, decrease, positiveCorrelation, negativeCorrelation, analogous, association, biomarkerFor, causesNoChange, directlyDecreases, directlyIncreases, hasActivity, hasComponent, hasComponents, hasMember, hasMembers, increases, isA, orthologous, prognosticBiomarkerFor, rateLimitingStepOf, regulates, subProcessOf, transcribedTo, translatedTo.\n- Please provide the support evidence by including the sentence that contains the extracted BEL triple.\n- Use  Namespaces to unambiguously reference concepts. \n- Associate prefix HGNC for human loci, including protein coding genes, ncRNA genes and pseudogenes.\n- Associate prefix CHEBI for molecular entities focused on small chemical compounds.\n- Associate prefix GO to unify the representation of gene and gene product attributes across all species.\n- Associate prefix MeSH for indexing articles for PubMed.\n- Associate prefix miRBase which is a biological database that acts as an archive of microRNA sequences and annotations before miRNA names.\n- Associate prefix MGI which is official gene symbol for a mouse gene.\n- Associate prefix DO as a standardized ontology for human disease.\n- Before each subject or object, use one of the BEL functions from the following: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n\nExample Triple:\nTriple1:\nsubject: biologicalProcess(GO:"response to fluid shear stress")\nrelation: increases\nobject: rnaAbundance(HGNC:NOS3)\nsupport evidence: NO is generated by endothelial NO synthase (eNOS, or NOS3), the activity of which is increased by fluid shear stress through various mechanisms.\n(Note: Ensure that the extracted BEL triples adhere to the specified format and namespaces.)'
} ]
      temperature = 0
      abstract = abstracts[pmid]
      message = "Extract all BEL triples for each sentence and provide support evidence for each triple from the following :" + abstract            
      if message:
            messages.append(
            {"role": "user", "content": message},)
            chat = openai.ChatCompletion.create(top_p = 0.1, model="gpt-4", messages=messages, )
            ''''''
      #model="gpt-3.5-turbo"
      reply = chat.choices[0].message.content
      print(reply)
      file = open(str(pmid)+"prompt3GPT4"+"top_p0.1-"+str(i)+".txt", "w", encoding='utf-8')
      file.write(reply)
      file.close()
      messages.append({"role": "assistant", "content": reply})
   except Exception as e:
      print(e)

 10%|█         | 1/10 [00:37<05:41, 37.89s/it]

Triple1:
subject: proteinAbundance(HGNC:MAPT)
relation: association
object: pathology(DO:"Alzheimer's disease")
support evidence: Hyperphosphorylated forms of the neuronal microtubule (MT)-associated protein tau are major components of Alzheimer's disease paired helical filaments.

Triple2:
subject: proteinAbundance(HGNC:PPP2CA)
relation: directlyIncreases
object: proteinAbundance(HGNC:MAPT)
support evidence: Previously, we reported that ABalphaC, the dominant brain isoform of protein phosphatase 2A (PP2A), is localized on MTs, binds directly to tau, and is a major tau phosphatase in cells.

Triple3:
subject: proteinAbundance(HGNC:PPP2CA)
relation: association
object: proteinAbundance(HGNC:MAPT)
support evidence: We now describe direct interactions among tau, PP2A, and MTs at the submolecular level.

Triple4:
subject: proteinAbundance(HGNC:PPP2CA)
relation: directlyDecreases
object: proteinAbundance(HGNC:MAPT)
support evidence: Specific PP2A isoforms bind to MTs with distinct affinitie

In [3]:
with open('abstracts.pkl', 'rb') as f:
    abstracts = pickle.load(f)

print(len(abstracts))

250


In [ ]:
#Run on the whole corpora of Tau
with open('abstracts-checked.pkl', 'rb') as f:
    abstracts = pickle.load(f)
cwd = os.getcwd()
for i in range(5,6):
    for pmid in tqdm(abstracts.keys()):
        if os.path.exists('chatgpt-responses/gpt-4/Prompt-2/setting-1/trial-{i_}'.format(i_=i)+'/'+str(pmid)+'.txt'): #Skip already processed docs
            print(" >>>> skipping", pmid, '...')
        else:
            try:
                messages = [ {"role": "system",
    "content": 'Extract BEL Triples:\n- Your task is to extract Biological Expression Language (BEL) triples from provided text.\n- A BEL triple consists of three components: subject, relation, and object.\nThe subject and object should be entities categorized into specific namespaces, such as MESH for diseases or biological processes, HGNC for genes, GO for gene functions, and CHEBI for chemicals.\n- Each subject or object should be preceded by one of the BEL functions: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n- The relation between subject and object should be chosen from the following list: increase, decrease, positiveCorrelation, negativeCorrelation, analogous, association, biomarkerFor, causesNoChange, directlyDecreases, directlyIncreases, hasActivity, hasComponent, hasComponents, hasMember, hasMembers, increases, isA, orthologous, prognosticBiomarkerFor, rateLimitingStepOf, regulates, subProcessOf, transcribedTo, translatedTo.\n- Please provide the support evidence by including the sentence that contains the extracted BEL triple.\n- Use  Namespaces to unambiguously reference concepts. \n- Associate prefix HGNC for human loci, including protein coding genes, ncRNA genes and pseudogenes.\n- Associate prefix CHEBI for molecular entities focused on small chemical compounds.\n- Associate prefix GO to unify the representation of gene and gene product attributes across all species.\n- Associate prefix MeSH for indexing articles for PubMed.\n- Associate prefix miRBase which is a biological database that acts as an archive of microRNA sequences and annotations before miRNA names.\n- Associate prefix MGI which is official gene symbol for a mouse gene.\n- Associate prefix DO as a standardized ontology for human disease.\n- Before each subject or object, use one of the BEL functions from the following: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n\nExample Triple:\nTriple1:\nsubject: biologicalProcess(GO:"response to fluid shear stress")\nrelation: increases\nobject: rnaAbundance(HGNC:NOS3)\nsupport evidence: NO is generated by endothelial NO synthase (eNOS, or NOS3), the activity of which is increased by fluid shear stress through various mechanisms.\n(Note: Ensure that the extracted BEL triples adhere to the specified format and namespaces.)'} ]
                time.sleep(2)
                # temperature = 0  (using default)
                top_p = 0.1
                abstract = abstracts[pmid]
                message = "Extract all BEL triples for each sentence and provide support evidence for each triple from the following :" + abstract         
                if message:
                    messages.append(
                    {"role": "user", "content": message},)
                    chat = openai.ChatCompletion.create(top_p = .1, model="gpt-4", messages=messages, )
                    ''''''
                #model="gpt-3.5-turbo"
                #model = "gpt-4"
                reply = chat.choices[0].message.content
                #print(reply)
                file = open(cwd+"/chatgpt-responses/gpt-4/Prompt-2/setting-1/trial-{i_}/".format(i_=i)+str(pmid)+".txt", "w+", encoding='utf-8')
                file.write(reply)
                file.close()
                messages.append({"role": "assistant", "content": reply})
                #time.sleep(5)
            except Exception as e:
                time.sleep(2)
                print(e)

In [ ]:
#Run on the whole full texts chunks provided by Juergen
path_cwd = str(os.getcwd())
path = path_cwd+"/fulltext-chunked-Juergen/"
print(path)
file_list = glob.glob(path + "/*.csv")
excl_list = []
all_triples = []
not_compiled = []
for file in tqdm(file_list):
    #print(file)
    df = pd.read_csv(file)
    #print(df)
    # for col in excel_part.columns:
    #     print(col)
    for index, row in df.iterrows():
        pmcid = row["pmcid"]
        chunk = row["chunk"]
        if os.path.exists(path_cwd+'/chatgpt-responses-fulltext/gpt4/trial4/{pmcid_}'.format(pmcid_ = pmcid)+'.txt'): #Skip already processed docs
            print(" >>>> skipping", pmcid, '...')
        else:
            try:
                messages = [ {"role": "system",
    "content": 'Extract BEL Triples:\n- Your task is to extract Biological Expression Language (BEL) triples from provided text.\n- A BEL triple consists of three components: subject, relation, and object.\nThe subject and object should be entities categorized into specific namespaces, such as MESH for diseases or biological processes, HGNC for genes, GO for gene functions, and CHEBI for chemicals.\n- Each subject or object should be preceded by one of the BEL functions: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n- The relation between subject and object should be chosen from the following list: increase, decrease, positiveCorrelation, negativeCorrelation, analogous, association, biomarkerFor, causesNoChange, directlyDecreases, directlyIncreases, hasActivity, hasComponent, hasComponents, hasMember, hasMembers, increases, isA, orthologous, prognosticBiomarkerFor, rateLimitingStepOf, regulates, subProcessOf, transcribedTo, translatedTo.\n- Please provide the support evidence by including the sentence that contains the extracted BEL triple.\n- Use  Namespaces to unambiguously reference concepts. \n- Associate prefix HGNC for human loci, including protein coding genes, ncRNA genes and pseudogenes.\n- Associate prefix CHEBI for molecular entities focused on small chemical compounds.\n- Associate prefix GO to unify the representation of gene and gene product attributes across all species.\n- Associate prefix MeSH for indexing articles for PubMed.\n- Associate prefix miRBase which is a biological database that acts as an archive of microRNA sequences and annotations before miRNA names.\n- Associate prefix MGI which is official gene symbol for a mouse gene.\n- Associate prefix DO as a standardized ontology for human disease.\n- Before each subject or object, use one of the BEL functions from the following: abundance, activity, biologicalProcess, pathology, proteinAbundance, variant.\n\nExample Triple:\nTriple1:\nsubject: biologicalProcess(GO:"response to fluid shear stress")\nrelation: increases\nobject: rnaAbundance(HGNC:NOS3)\nsupport evidence: NO is generated by endothelial NO synthase (eNOS, or NOS3), the activity of which is increased by fluid shear stress through various mechanisms.\n(Note: Ensure that the extracted BEL triples adhere to the specified format and namespaces.)'} ]
                time.sleep(2)
                # temperature = 0  (using default)
                top_p = 0.1
                text = chunk
                message = "Extract all BEL triples for each sentence and provide support evidence for each triple from the following :" + text         
                if message:
                    messages.append(
                    {"role": "user", "content": message},)
                    chat = openai.ChatCompletion.create(top_p = .1, model="gpt-4", messages=messages, )
                    ''''''
                #model="gpt-3.5-turbo"
                #model = "gpt-4"
                reply = chat.choices[0].message.content
                #print(reply)
                file = open(path_cwd+"/chatgpt-responses-fulltext/gpt4/trial4/{pmcid_}".format(pmcid_ = pmcid)+".txt", "w+", encoding='utf-8')
                file.write(reply)
                file.close()
                messages.append({"role": "assistant", "content": reply})
                time.sleep(5)
            except Exception as e:
                time.sleep(5)
                print(e)
        

In [19]:
import os
os.getcwd()

'/Users/neginbabaiha/Documents/GitHub/chatgpt-paper'